1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php).
2. Сделать feature engineering.
3. Обучить любой классификатор (какой вам нравится).
4. Далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть.
5. Применить random negative sampling для построения классификатора в новых условиях.
6. Сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик).

In [145]:
import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

import xgboost as xgb

Для модели возьмем набор для прогнозирования дохода на основе данных перепеси- https://archive.ics.uci.edu/ml/datasets/Adult .

In [146]:
df = pd.read_csv(
    'adult.data', sep=',', index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
           'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

In [147]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Проведем Feature Engineering.

Проверим наличие пропусков.

In [148]:
pd.DataFrame([df.isna().sum(), df.isnull().sum()], index={'na', 'null'})

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
na,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Преобразуем значения целевой переменной 'income' к значениям 0 или 1.

In [149]:
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})

Воспользуемся One-Hot Encoding для кодирования признаков, кроме целевой переменной.

In [150]:
X = pd.get_dummies(df.iloc[:, :-1])

In [151]:
y = df['income']

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Обучим модель.

In [153]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

C:\Users\79609\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:27:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [154]:
def evaluate_results(y_test, y_predict):

    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

In [155]:
evaluate_results(y_test, y_predict)

Classification results:
f1: 70.62%
roc: 79.77%
recall: 66.08%
precision: 75.84%


Разделим набор на два множества- P (positives) и U (unlabeled). Для обучения возьмем часть положительных классов, а остальную часть и unlabeled будем считать неизвестными.

In [156]:
mod_data = pd.get_dummies(df.copy())
mod_data['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

Получим показатели положительных проб.

In [157]:
pos_ind = np.where(mod_data.loc[:, 'income'].values == 1)[0]

Проведем перемешивание.

In [158]:
np.random.shuffle(pos_ind)

Оставим четверть положительных отзывов.

In [159]:
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Используем {pos_sample_len}/{len(pos_ind)} .')
pos_sample = pos_ind[:pos_sample_len]

Используем 1961/7841 .


Создадим столбец для новой целевой переменной, в которой будут два класса- P (1) и U (-1).

In [160]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:, -1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


In [161]:
x_data = mod_data.drop(['income', 'class_test'], axis=1).values 
y_labeled = mod_data.loc[:, 'class_test'].values 
y_positive = mod_data.loc[:, 'income'].values 

Применим random negative sampling.

In [162]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test'] == -1][:len(mod_data[mod_data['class_test'] == 1])]
sample_test = mod_data[mod_data['class_test'] == -1][len(mod_data[mod_data['class_test'] == 1]):]
pos_sample = mod_data[mod_data['class_test'] == 1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [163]:
model_rns = xgb.XGBClassifier()
model.fit(sample_train.drop(['income', 'class_test'], axis=1).values,
          sample_train.loc[:, 'income'].values)
y_predict_rns = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)

[23:27:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\79609\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Classification results:
f1: 60.94%
roc: 82.18%
recall: 88.56%
precision: 46.46%


Сравним полученные метрики.

In [164]:
def evaluate_results_(y_test, y_predict):
   
    f1 = f1_score(y_test, y_predict)
    
    roc = roc_auc_score(y_test, y_predict)
    
    recall = recall_score(y_test, y_predict)
    
    precision = precision_score(y_test, y_predict)
    
    return f1, roc, recall, precision

In [165]:
pd.DataFrame([
    evaluate_results_(y_test, y_predict),
    evaluate_results_(sample_test.loc[:, 'income'].values, y_predict_rns)
], columns={'f1', 'roc', 'recall', 'precision'}, index={'Random negative sampling', 'Simple xgboost'}).round(2)

,recall,f1,roc,precision
Random negative sampling,0.71,0.80,0.66,0.76
Simple xgboost,0.61,0.82,0.89,0.46
